In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install fastapi nest_asyncio uvicorn pyngrok diffusers transformers torch accelerate python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from unsloth import FastLanguageModel
import torch
import os

# ✅ Auto-detect best dtype
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# ✅ Use 4-bit quantization if on GPU (reduces VRAM usage)
load_in_4bit = torch.cuda.is_available()

# ✅ Load the fine-tuned model and tokenizer
model_name = "sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=os.getenv("HUGGINGFACE_TOKEN"),
)

# ✅ Ensure model is on the correct device
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Confirm everything is set correctly
print(f"Model loaded on {device} with dtype={dtype} (4-bit={load_in_4bit})")
FastLanguageModel.for_inference(model)

In [ ]:
import torch
import re
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from unsloth import FastLanguageModel
import uvicorn
import os
import nest_asyncio
from pyngrok import ngrok
import asyncio
from threading import Thread # Import Thread

# Apply nest_asyncio to allow running asyncio event loop in environments like notebooks
nest_asyncio.apply()

# ----------------------------
# Helper Functions
# ----------------------------

def fix_spacing(text):
    """Fix missing spaces in Urdu text."""
    # Using the updated regex for Urdu characters
    return re.sub(r'(?<=[؀-ۿ])(?=[؀-ۿ])', ' ', text)

def extract_text_after_last_story(text: str) -> str:
    """
    Extracts text after the last occurrence of "Story:" and ensures it ends with "۔"
    """
    matches = [m.end() for m in re.finditer(r'(?i)Story:', text)]
    if matches:
        extracted_text = text[matches[-1]:].strip()
        # Using the correct Urdu full stop
        last_full_stop = extracted_text.rfind("۔")
        if last_full_stop != -1:
            return extracted_text[:last_full_stop + 1].strip()
    return ""

def remove_duplicate_sentences(text: str) -> str:
    """Removes duplicate sentences from the text based on the Urdu full stop '۔'."""
    # Using the correct Urdu full stop
    sentences = text.split("۔")
    seen = set()
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence and sentence not in seen:
            seen.add(sentence)
            cleaned_sentences.append(sentence)
    # Using the correct Urdu full stop for joining
    return "۔ ".join(cleaned_sentences) + "۔" if cleaned_sentences else ""

# ----------------------------
# Story Generation Function
# ----------------------------


def generate_story_outline(concept: str, initial_story: str = "", max_steps: int = 9) -> str:
    """
    Generates a structured, coherent, and grammatically sound Urdu story iteratively.
    Uses a step-wise template to build narrative depth, with explicit instructions to ensure
    a complete and engaging journey that concludes definitively.
    """
    if model is None or tokenizer is None or device is None:
        return "Model or tokenizer not loaded. Cannot generate story. Please check your environment setup and model paths/permissions."

    story_text = initial_story
    complete_story = initial_story

    for step in range(1, max_steps + 1):
        if step == 1:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
[🔹 آغاز کریں:
- جملے چھوٹے اور براہ راست ہوں۔
- کہانی میں اسرار اور دلچسپی پیدا کریں۔
- صرف اردو میں کہانی شروع کریں۔]

📜 **براہ کرم کہانی کا دلکش آغاز کریں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 2:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 کہانی کو آگے بڑھائیں:
- کردار کے ابتدائی ارادوں اور چھپے رازوں کو نمایاں کریں۔
- کہانی میں مزید سوالات اور اسرار پیدا کریں۔]

📜 **براہ کرم کہانی کو ایک نیا موڑ دیں اور گہرائی پیدا کریں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 3:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 تناؤ اور کشمکش کو بڑھائیں:
- کردار کی داخلی کشمکش اور غیر متوقع موڑ کو اجاگر کریں۔
- کہانی میں پیچیدگی اور دلچسپی پیدا کریں۔]

📜 **براہ کرم کہانی کو ایک ایسے مقام پر لے جائیں جہاں قاری حیران رہ جائے۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 4:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 عروج کی طرف بڑھیں:
- کہانی میں سنسنی خیزی اور نئے انکشافات شامل کریں۔
- کرداروں کی جدوجہد اور مقابلے کو واضح کریں۔]

📜 **براہ کرم کہانی کو ایک عروج پر پہنچائیں جہاں ہر لمحہ نیا انکشاف ہو۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 5:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 موڑ اور نیا رخ:
- کہانی کو ایک نئے اور غیر متوقع موڑ پر لے جائیں۔
- مزاحمت اور چیلنجز کو اجاگر کریں۔]

📜 **براہ کرم کہانی میں نیا رخ اور مزید کشمکش شامل کریں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 6:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 اختتامی مراحل کی طرف بڑھیں:
- کہانی کو مزید تفصیل سے بیان کریں اور چیلنجز کی شدت بڑھائیں۔
- کردار کی جدوجہد کو گہرائی سے پیش کریں۔]

📜 **براہ کرم کہانی کو اختتامی مراحل کی طرف لے جائیں، مگر ایک آخری حیران کن موڑ چھوڑیں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 7:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 مکمل اختتام کی تیاری:
- کہانی کو ایک مربوط اور مکمل انجام کی طرف لے جائیں۔
- تمام اہم موڑ اور کشمکش کو حل کریں۔]

📜 **براہ کرم کہانی کو شاندار انجام تک پہنچائیں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 8:
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 جزئیات کی ترتیب اور اختتام:
- کہانی کو مکمل کریں اور آخری تاثرات چھوڑیں۔
- غیر ضروری تفصیلات کو حذف کرتے ہوئے مرکزی کہانی پر توجہ دیں۔]

📜 **براہ کرم کہانی کو ایک واضح اور مکمل انجام کے ساتھ ختم کریں۔**
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """
        elif step == 9:  # Final polishing step with lower temperature for coherence
            template = f"""
**💡 بنیادی خیال:** {concept}

کہانی:
{story_text}

[🔹 براہ کرم کہانی کو ایک مکمل، مربوط اور دلکش انجام تک پہنچائیں:
- کہانی کے تمام اہم موڑ اور کشمکش کو حل کریں۔
- کردار کی ذہنی اور جذباتی ترقی کو اجاگر کریں۔
- روزمرہ کی تفصیلات سے ہٹ کر ایک دلچسپ سفر اور تبدیلی کو مرکزی حیثیت دیں۔
- کہانی ایک واضح، اثر انگیز اختتام پر ختم ہو۔]

📜 **براہ کرم کہانی کو مکمل اور مربوط اختتام کے ساتھ ختم کریں۔**
صرف اردو میں کہانی لکھیں اور اضافی ہدایات شامل نہ کریں۔
After this dont write indicators for users just Start the story in Urdu only after the word "Story:"
            """

        # Prepare input tokens from the template
        inputs = tokenizer(template, return_tensors="pt").to(device)

        # Adjust max tokens and temperature per step
        max_tokens = 250
        temperature = 0.7
        if step >= 7:
            max_tokens = 400
        if step == 9:
            max_tokens = 500
            temperature = 0.6

        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            return_dict_in_generate=True
        )

        # Extract the new text generated after the last occurrence of "Story:"
        new_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
        extracted_text = extract_text_after_last_story(new_text)

        # Update story text for the next step
        story_text = extracted_text
        complete_story += " " + extracted_text

    complete_story = remove_duplicate_sentences(complete_story)
    return complete_story

app = FastAPI()

class StoryRequest(BaseModel):
    concept: str
    initial_story: str = ""
    max_steps: int = 9

@app.post("/generate_story/")
async def generate_story(request: StoryRequest):
    try:
        story = generate_story_outline(request.concept, request.initial_story, request.max_steps)
        return {"story": story}
    except Exception as e:
        # Log the exception for debugging
        print(f"An error occurred: {e}")
        raise HTTPException(status_code=500, detail=f"An error occurred during story generation: {e}")

# Code to run the FastAPI app with uvicorn and expose with ngrok
# This pattern is suitable for environments like Google Colab or Jupyter notebooks
if __name__ == "__main__":
    # Set the ngrok authtoken from environment variable
    # Make sure to set the NGROK_AUTH_TOKEN environment variable
    ngrok_auth_token = os.environ.get("NGROK_AUTH_TOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Warning: NGROK_AUTH_TOKEN environment variable not set. ngrok might not work.")
        print("You can set it like this: export NGROK_AUTH_TOKEN='YOUR_NGROK_AUTH_TOKEN' (Linux/macOS) or set NGROK_AUTH_TOKEN='YOUR_NGROK_AUTH_TOKEN' (Windows)")


    # Define the port for FastAPI
    port = 8000

    # Start ngrok tunnel
    try:
        print(f"Starting ngrok tunnel for port {port}...")
        # Use bind_tls=True if you need HTTPS, but stick to False for simplicity if not required
        public_url = ngrok.connect(port, "http", bind_tls=False).public_url
        print(f" FastAPI app is exposed at: {public_url}")
        print(f"Access the FastAPI docs at: {public_url}/docs")
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
        print("Please ensure ngrok is installed, authenticated, and not already running on this port.")
        public_url = None # Set to None if ngrok fails

    # Run the FastAPI application using uvicorn in a separate thread
    # Only start the thread if ngrok started successfully or if you intend to access locally
    if public_url or not ngrok_auth_token: # Added condition to allow local run without ngrok token
        try:
            print("Starting Uvicorn server in a separate thread...")
            # Use reload=False when running with ngrok to avoid issues
            # daemon=True allows the thread to exit when the main program exits
            uvicorn_thread = Thread(target=uvicorn.run,
                                    kwargs={"app": app, "host": "0.0.0.0", "port": port, "reload": False},
                                    daemon=True)
            uvicorn_thread.start()
            print("Uvicorn server thread started. The script can now continue.")
            # If running as a script, you might want to add a line here to keep the main thread alive,
            # e.g., input("Press Enter to stop the server...\n") or a loop
            # In Colab, the notebook environment keeps the script alive.
        except Exception as e:
            print(f"Error starting uvicorn thread: {e}")
            print("Please check if the port is already in use.")
    else:
        print("ngrok failed to start. Uvicorn server will not be started automatically.")

You can set it like this: export NGROK_AUTH_TOKEN='YOUR_NGROK_AUTH_TOKEN' (Linux/macOS) or set NGROK_AUTH_TOKEN='YOUR_NGROK_AUTH_TOKEN' (Windows)
Starting ngrok tunnel for port 8000...
 FastAPI app is exposed at: http://3576-34-87-126-232.ngrok-free.app
Access the FastAPI docs at: http://3576-34-87-126-232.ngrok-free.app/docs
Starting Uvicorn server in a separate thread...
Uvicorn server thread started. The script can now continue.


In [ ]:
# import requests
# import json

# url = "http://3cfe-34-125-125-99.ngrok-free.app/generate_story/"

# payload = {
#     "concept": "ایسی کہانی لکھیں جس میں ایک شخص انتہائی غریب ہوتا ہے۔ وہ اپنی زندگی امیر بننے کے لیے وقف کر دیتا ہے، لیکن جب وہ دولت مند بن جاتا ہے تو اسے احساس ہوتا ہے کہ اس نے اپنی زندگی میں کیا کھو دیا ہے۔",
#     "initial_story": "",
#     "max_steps": 9
# }

# headers = {
#     "Content-Type": "application/json"
# }

# try:
#     response = requests.post(url, data=json.dumps(payload), headers=headers)
#     response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
#     print("Response Status Code:", response.status_code)
#     print("Response Body:", response.json())
# except requests.exceptions.RequestException as e:
#     print(f"An error occurred during the request: {e}")